In [ ]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2022-02-24 08:27:45--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.186.54, 104.21.92.44, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.186.54|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6532197 (6.2M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.23M  7.69MB/s    in 0.8s    

2022-02-24 08:27:46 (7.69 MB/s) - ‘fra-eng.zip’ saved [6532197/6532197]



In [ ]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
# Import Libraries
from __future__ import print_function
from keras.models import Model
from keras.layers import Input,LSTM ,Dense
import numpy as np

In [ ]:
batch_s = 64 #batch size for training
epochs = 100 # No. of epochs to train for
latent_dimension = 256 # Latent dimensionalityof the encoding space
number_samples = 10000 # Number of samples to train on
# Path to data text file on disk
data_path = "fra.txt"

In [ ]:
import pdb

In [ ]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()
with open(data_path,'r',encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[:min(number_samples,len(lines)-1)]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t'+target_text+'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_chars:
      input_chars.add(char)
  for char in target_text:
    if char not in target_chars:
      target_chars.add(char)

input_characters = sorted(list(input_chars))
target_characters = sorted(list(target_chars))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_sequence_length = max([len(text) for text in input_texts])
max_decoder_sequence_length = max([len(text) for text in target_texts])

In [ ]:
print("Input characters: ",input_characters)
print('\n')
print("Target characters: ",target_characters)
print('\n')
print("Input texts:",input_texts[0:2])
print('\n')
print("Target texts:",target_texts[0:2])
print('\n')
print("No. of chars from encoding language: ",num_encoder_tokens)
print('\n')
print("No. of chars to decoded language: ",num_decoder_tokens)
print('\n')
print("Max length of text in encoding language: ",max_encoder_sequence_length)
print('\n')
print("Max length of text in decoding language: ",max_decoder_sequence_length)

Input characters:  [' ', '!', '"', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'é']


Target characters:  ['\t', '\n', ' ', '!', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']


Input texts: ['Go.', 'Go.']


Target texts: ['\tVa !\n', '\tMarche.\n']


No. of chars from encoding language:  71


No. of c

In [ ]:
input_token_index = dict(
    [(char,i) for i,char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i,char in enumerate(target_characters)]
)

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_sequence_length,num_encoder_tokens),# matrix shape: rows=no. of texts to be passed, columns=max length needed to encode , layers=each one for each character
    dtype='float32'
)

decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_sequence_length,num_decoder_tokens),
    dtype='float32'
)

decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_sequence_length,num_decoder_tokens),
    dtype='float32'
)

In [ ]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
print(encoder_input_data.shape)

(10000, 15, 71)


In [ ]:
print(decoder_input_data.shape)

(10000, 59, 92)


In [ ]:
print(decoder_target_data.shape)

(10000, 59, 92)


In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t:,input_token_index[char]] = 1
  encoder_input_data[i,t+1:,input_token_index[' ']] = 1
  for t,char in enumerate(target_text):
    decoder_input_data[i,t:,target_token_index[char]] = 1
    if t>0:
      decoder_target_data[i,t-1:,target_token_index[char]] = 1
  decoder_input_data[i,t+1: ,target_token_index[' ']] = 1
  decoder_target_data[i,t:,target_token_index[' ']] = 1

In [ ]:
input_token_index['G']

26

In [ ]:
encoder_input_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [ ]:
encoder_input_data[0][0][26] # G from 'Go' is converted to above vector

1.0

In [ ]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = LSTM(latent_dimension,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None,num_decoder_tokens))
decoder_lstm = LSTM(latent_dimension, return_sequences=True,return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs] , decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit([encoder_input_data, decoder_input_data] ,
          decoder_target_data,
          batch_size = batch_s,
          epochs = epochs,
          validation_split = 0.2)

Epoch 1/100
125/125 [==============================] - 53s 403ms/step - loss: 49.3891 - accuracy: 0.2090 - val_loss: 57.1404 - val_accuracy: 0.2461
Epoch 2/100
125/125 [==============================] - 45s 356ms/step - loss: 57.5323 - accuracy: 0.2140 - val_loss: 66.6795 - val_accuracy: 0.2489
Epoch 3/100
125/125 [==============================] - 44s 350ms/step - loss: 65.9868 - accuracy: 0.2151 - val_loss: 76.7105 - val_accuracy: 0.2489
Epoch 4/100
125/125 [==============================] - 45s 358ms/step - loss: 74.6945 - accuracy: 0.2149 - val_loss: 86.6214 - val_accuracy: 0.2488
Epoch 5/100
125/125 [==============================] - 43s 346ms/step - loss: 83.2935 - accuracy: 0.2147 - val_loss: 96.9463 - val_accuracy: 0.2487
Epoch 6/100
125/125 [==============================] - 44s 348ms/step - loss: 92.2532 - accuracy: 0.2145 - val_loss: 107.0755 - val_accuracy: 0.2474
Epoch 7/100
125/125 [==============================] - 43s 348ms/step - loss: 101.2305 - accuracy: 0.2131 - val

In [ ]:
model.save('seq2seq.h5')